### **title 기반 Semantic Alignment(의미적 일치성) 설계**

**Semantic Alignment (의미적 일치성 - Vibe Check)**

- **Goal:** 사용자의 의도(Taxonomy)와 추천된 '노래의 분위기/가사'가 일치하는가?
- **Primary KPI:** LLM Judge에게 "이 상황(독서실)에 이 노래(Bang Bang)가 어울리는가?"를 묻고 **평균 4.5점** 달성.
- **Guardrail KPI:** 상황과 정반대되는 노래(예: 수면에 락, 이별 위로에 신나는 댄스) 추천 발생률 **0% 유지**.

평가 로직: LLM Judge가 "추천된 노래 제목"과 "GPT의 추천 사유(Reasoning)"를 보고, LLM의 지식을 기반으로 이 상황에 맞는지 5점 만점으로 채점합니다.

평가셋 구축

In [1]:
import pandas as pd

# 20개의 테스트 케이스 정의 (Taxonomy 기반)
data = [
    # 1. [기본] 난이도 하 (User Pref 없음, 상황에 충실)
    {"ID": 1, "Location": "library", "Decibel": "silent", "Goal": "focus", "User Pref (Genre/Artist)": "None", "Expected Keywords / Vibe (기대 결과)": "piano, white noise, study beats, classical, calm", "Type": "기본", "난이도": "하"},
    {"ID": 2, "Location": "gym", "Decibel": "loud", "Goal": "active", "User Pref (Genre/Artist)": "None", "Expected Keywords / Vibe (기대 결과)": "workout remix, edm, high tempo, gym motivation, energetic", "Type": "기본", "난이도": "하"},
    {"ID": 3, "Location": "home", "Decibel": "quiet", "Goal": "sleep", "User Pref (Genre/Artist)": "None", "Expected Keywords / Vibe (기대 결과)": "lullaby, rain sounds, ambient, meditation, deep sleep", "Type": "기본", "난이도": "하"},
    
    # 2. [취향 반영] 난이도 중 (상황과 취향이 잘 어울림)
    {"ID": 4, "Location": "cafe", "Decibel": "moderate", "Goal": "relax", "User Pref (Genre/Artist)": "Jazz", "Expected Keywords / Vibe (기대 결과)": "vocal jazz, coffee shop vibe, acoustic jazz, saxophone", "Type": "취향반영", "난이도": "중"},
    {"ID": 5, "Location": "park", "Decibel": "moderate", "Goal": "active", "User Pref (Genre/Artist)": "K-Pop (NewJeans)", "Expected Keywords / Vibe (기대 결과)": "upbeat k-pop, newjeans hype boy, refreshing pop, bright energy", "Type": "취향반영", "난이도": "중"},
    {"ID": 6, "Location": "co-working", "Decibel": "quiet", "Goal": "focus", "User Pref (Genre/Artist)": "Lo-fi", "Expected Keywords / Vibe (기대 결과)": "lo-fi beats, chillhop, no lyrics, productive vibe", "Type": "취향반영", "난이도": "중"},
    {"ID": 7, "Location": "moving", "Decibel": "loud", "Goal": "neutral", "User Pref (Genre/Artist)": "Pop", "Expected Keywords / Vibe (기대 결과)": "trendy pop, billboard hits, daily drive, rhythmic", "Type": "취향반영", "난이도": "중"},
    {"ID": 8, "Location": "home", "Decibel": "silent", "Goal": "consolation", "User Pref (Genre/Artist)": "Ballad (IU)", "Expected Keywords / Vibe (기대 결과)": "emotional ballad, iu slow songs, piano ballad, sad touching", "Type": "취향반영", "난이도": "중"},

    # 3. [충돌 해결] 난이도 상 (상황과 취향이 정반대 -> 타협점 찾아야 함) ★핵심★
    {"ID": 9, "Location": "library", "Decibel": "silent", "Goal": "focus", "User Pref (Genre/Artist)": "Heavy Metal", "Expected Keywords / Vibe (기대 결과)": "post-rock, instrumental rock, calm guitar, ambient metal (No screaming)", "Type": "충돌해결", "난이도": "상"},
    {"ID": 10, "Location": "gym", "Decibel": "loud", "Goal": "active", "User Pref (Genre/Artist)": "Classical", "Expected Keywords / Vibe (기대 결과)": "epic orchestral, cinematic battle music, fast tempo classical (High energy)", "Type": "충돌해결", "난이도": "상"},
    {"ID": 11, "Location": "home", "Decibel": "quiet", "Goal": "sleep", "User Pref (Genre/Artist)": "Hip-hop", "Expected Keywords / Vibe (기대 결과)": "lo-fi hip hop, chill beats, instrumental hip hop, slow jam (No rap)", "Type": "충돌해결", "난이도": "상"},
    {"ID": 12, "Location": "co-working", "Decibel": "quiet", "Goal": "focus", "User Pref (Genre/Artist)": "K-Pop (Dance)", "Expected Keywords / Vibe (기대 결과)": "k-pop piano cover, soft instrumental version, acoustic k-pop", "Type": "충돌해결", "난이도": "상"},
    {"ID": 13, "Location": "cafe", "Decibel": "moderate", "Goal": "relax", "User Pref (Genre/Artist)": "EDM", "Expected Keywords / Vibe (기대 결과)": "tropical house, chill house, downtempo, lounge music (Not festival big room)", "Type": "충돌해결", "난이도": "상"},
    {"ID": 14, "Location": "park", "Decibel": "moderate", "Goal": "consolation", "User Pref (Genre/Artist)": "Rock", "Expected Keywords / Vibe (기대 결과)": "soft rock, acoustic rock, britpop, warm vibe, modern rock ballad", "Type": "충돌해결", "난이도": "상"},
    
    # 4. [특수 상황] 소음 대응 및 감정 분석
    {"ID": 15, "Location": "subway", "Decibel": "very_loud", "Goal": "focus", "User Pref (Genre/Artist)": "Classical", "Expected Keywords / Vibe (기대 결과)": "cello solo, deep organ, steady rhythm (Noise masking strategy)", "Type": "소음대응", "난이도": "상"},
    {"ID": 16, "Location": "home", "Decibel": "moderate", "Goal": "anger", "User Pref (Genre/Artist)": "Rock", "Expected Keywords / Vibe (기대 결과)": "grunge, punk rock, hard rock, aggressive energy, stress relief", "Type": "감정분석", "난이도": "중"},
    {"ID": 17, "Location": "home", "Decibel": "moderate", "Goal": "anger", "User Pref (Genre/Artist)": "Classical", "Expected Keywords / Vibe (기대 결과)": "dramatic classical, wagner, stravinsky, intense orchestral", "Type": "감정분석", "난이도": "상"},
    {"ID": 18, "Location": "moving", "Decibel": "loud", "Goal": "active", "User Pref (Genre/Artist)": "Hip-hop", "Expected Keywords / Vibe (기대 결과)": "boom bap, rhythmic rap, running beats, energetic hip-hop", "Type": "감정분석", "난이도": "중"},
    {"ID": 19, "Location": "cafe", "Decibel": "moderate", "Goal": "neutral", "User Pref (Genre/Artist)": "Indie", "Expected Keywords / Vibe (기대 결과)": "indie pop, acoustic indie, calm afternoon, easy listening", "Type": "일반", "난이도": "하"},
    {"ID": 20, "Location": "library", "Decibel": "silent", "Goal": "focus", "User Pref (Genre/Artist)": "Podcast (Talk)", "Expected Keywords / Vibe (기대 결과)": "instrumental bgm, minimal ambient (Ignore talk preference)", "Type": "충돌해결", "난이도": "상"}
]

# DataFrame 생성
df = pd.DataFrame(data)

# CSV 파일로 저장 (한글 깨짐 방지 utf-8-sig 사용)
file_name = "evaluation_set_v1.csv"
df.to_csv(file_name, index=False, encoding="utf-8-sig")

print(f"'{file_name}' 파일 생성 완료! (총 {len(df)}개 케이스)")
print("이제 평가 코드를 실행하시면 됩니다.")

'evaluation_set_v1.csv' 파일 생성 완료! (총 20개 케이스)
이제 평가 코드를 실행하시면 됩니다.


Primary KPI: LLM Judge 평균 4.5점 달성

In [ ]:
import pandas as pd
from openai import OpenAI
import json
import time

# ===========================================================
# 1. 설정 (Configuration)
# ===========================================================
API_KEY = "YOUR_KEY_HERE"  #  여기에 본인의 OpenAI API Key를 입력하세요!
client = OpenAI(api_key=API_KEY)

INPUT_FILE = "evaluation_set_v1.csv"            # 아까 만든 평가셋
OUTPUT_FILE = "evaluation_result_semantic_final.xlsx" # 최종 결과 파일

# ===========================================================
# 2. 프롬프트 설계 (최종 확정 스키마 반영)
# ===========================================================

# [DJ 역할] 백엔드에 전달한 최종 스키마 그대로 적용
SYSTEM_PROMPT_DJ = """
# Role
당신은 사용자의 상황(Context)과 취향(Preference)에 맞춰, Spotify에 실존하는 음악을 추천하는 AI DJ입니다.

# Critical Constraints
1. **Real Song Only**: 반드시 실제로 존재하는 곡을 추천해야 합니다. (할루시네이션 금지)
2. **Context Alignment**: 목표(Goal)와 소음(Decibel)을 최우선으로 고려하세요.
   - 예: Sleep 목표 -> 락/댄스 금지.
3. **Format**: 반드시 아래 JSON 포맷을 지키세요.

# Output JSON Structure
{
  "recommendation_meta": {
    "reasoning": "추천 이유 (한글 1문장)",
    "primary_tag": "mood_tag"
  },
  "track_info": {
    "artist_name": "Exact Artist Name",
    "track_id": "Exact Song Title Spotify ID",
    "track_title": "Exact Song Title"
  },
  "target_audio_features": {
    "min_tempo": int,
    "max_tempo": int,
    "target_energy": (float 0.0~1.0),
    "target_instrumentalness": (float 0.0~1.0)
    "target_valence": (float 0.0~1.0),
    "target_acousticness": (float 0.0~1.0)
  }
}
"""

# [Judge 역할] 노래 제목과 이유를 보고 적합성 평가
SYSTEM_PROMPT_JUDGE = """
# Role
당신은 음악 평론가이자 QA Judge입니다.
AI가 추천한 '노래(Song)'와 '추천 사유(Reasoning)'가 주어진 '상황(Context)'에 적합한지 평가합니다.

# Task
1. 추천된 노래(Artist - Title)가 실제로 존재하는지, 어떤 분위기인지 당신의 지식(Knowledge Base)을 통해 파악하세요.
2. 추천 사유(Reasoning)가 상황에 맞게 논리적인지 확인하세요.

# Scoring Criteria (1-5)
- 5점 (Perfect): 상황에 완벽하게 어울리는 명곡이며, 추천 사유도 논리적임.
- 3점 (Fair): 나쁘지 않으나, 베스트 초이스는 아님.
- 1점 (Fail): 상황과 정반대(분위기 파괴) 또는 존재하지 않는 노래(Hallucination).

# Output JSON
{"score": int, "reason": "string"}
"""

# ===========================================================
# 3. 함수 정의
# ===========================================================

def get_song_recommendation(row):
    """ [Step 1] GPT에게 최종 스키마로 추천 요청 """
    user_msg = f"""
    Location: {row['Location']}
    Decibel: {row['Decibel']}
    Goal: {row['Goal']}
    Preference: {row['User Pref (Genre/Artist)']}
    """
    try:
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": SYSTEM_PROMPT_DJ},
                {"role": "user", "content": user_msg}
            ],
            response_format={"type": "json_object"},
            temperature=0.7
        )
        return json.loads(response.choices[0].message.content)
    except Exception as e:
        print(f"❌ Gen Error: {e}")
        return {}

def evaluate_semantic_alignment(row, artist, title, reasoning, target_features):
    """ [Step 2] LLM Judge에게 의미적 일치성 평가 요청 """
    judge_msg = f"""
    [User Context]
    Location: {row['Location']}, Goal: {row['Goal']}, Decibel: {row['Decibel']}
    User Pref: {row['User Pref (Genre/Artist)']}
    
    [AI Recommendation]
    Song: {artist} - {title}
    Reasoning: {reasoning}
    Target Features (AI Intention): {target_features}
    
    [Expected Vibe]
    {row['Expected Keywords / Vibe (기대 결과)']}
    
    위 노래와 추천 사유가 이 상황에 적합한지 평가해주세요.
    """
    try:
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": SYSTEM_PROMPT_JUDGE},
                {"role": "user", "content": judge_msg}
            ],
            response_format={"type": "json_object"}
        )
        return json.loads(response.choices[0].message.content)
    except Exception as e:
        return {"score": 0, "reason": f"Error: {e}"}

# ===========================================================
# 4. 메인 실행
# ===========================================================
print("\n🚀 [Final Schema] 의미적 일치성 평가 시작...")
df = pd.read_csv(INPUT_FILE)
results = []

for index, row in df.iterrows():
    print(f"▶ Case {index + 1}: {row['Location']} ({row['Goal']})...", end=" ")
    
    # 1. 추천 생성
    gpt_output = get_song_recommendation(row)
    
    # 데이터 파싱 (에러 처리 포함)
    meta = gpt_output.get('recommendation_meta', {})
    track = gpt_output.get('track_info', {})
    target = gpt_output.get('target_audio_features', {})
    
    artist = track.get('artist_name', "Unknown")
    title = track.get('track_title', "Unknown")
    reasoning = meta.get('reasoning', "No reasoning")
    
    print(f"🎵 {artist} - {title}")
    
    # 2. 품질 평가 (LLM Judge)
    judge_result = evaluate_semantic_alignment(row, artist, title, reasoning, target)
    
    # 3. 저장
    results.append({
        "Case ID": row['ID'],
        "Scenario": f"{row['Location']} / {row['Goal']}",
        "Conflict Type": row['Type'],
        "Recommended Artist": artist,
        "Recommended Title": title,
        "Reasoning": reasoning,
        "Target Energy": target.get('target_energy', 0), # 분석용 저장
        "LLM Score": judge_result['score'],
        "Judge Reason": judge_result['reason']
    })
    
    time.sleep(0.5)

# ===========================================================
# 5. 결과 저장 및 KPI 확인
# ===========================================================
result_df = pd.DataFrame(results)
result_df.to_excel(OUTPUT_FILE, index=False)

avg_score = result_df['LLM Score'].mean()
fatal_errors = len(result_df[result_df['LLM Score'] == 1])

print("\n" + "="*50)
print(f"✅ 평가 완료! 파일: {OUTPUT_FILE}")
print(f"📊 [KPI 1] 의미적 일치성 평균 점수: {avg_score:.2f} / 5.0 (목표: 4.5)")
print(f"🛡️ [KPI 2] 치명적 오류(1점) 발생: {fatal_errors}건 (목표: 0건)")
print("="*50)


🚀 [Final Schema] 의미적 일치성 평가 시작...
▶ Case 1: library (focus)... 🎵 Max Richter - On the Nature of Daylight
▶ Case 2: gym (active)... 🎵 The Prodigy - Firestarter
▶ Case 3: home (sleep)... 🎵 Max Richter - Dream 3 (in the midst of my life)
▶ Case 4: cafe (relax)... 🎵 Miles Davis - Blue in Green
▶ Case 5: park (active)... 🎵 NewJeans - Hype Boy
▶ Case 6: co-working (focus)... 🎵 Idealism - Snowfall
▶ Case 7: moving (neutral)... 🎵 Dua Lipa - Don't Start Now
▶ Case 8: home (consolation)... 🎵 IU - Through the Night
▶ Case 9: library (focus)... 🎵 Animals as Leaders - CAFO
▶ Case 10: gym (active)... 🎵 Ludwig van Beethoven - Symphony No. 7 in A Major, Op. 92: II. Allegretto
▶ Case 11: home (sleep)... 🎵 Mac Miller - 2009
▶ Case 12: co-working (focus)... 🎵 BTS - Butter
▶ Case 13: cafe (relax)... 🎵 Kygo - Stole the Show
▶ Case 14: park (consolation)... 🎵 Foo Fighters - Walking After You
▶ Case 15: subway (focus)... 🎵 Ludovico Einaudi - Nuvole Bianche
▶ Case 16: home (anger)... 🎵 Rage Against the Machi

---

GPT가 키워드 생성이 아닌 제목을 생성하기 떄문에 임베딩 못함(임베딩 모델은 밤편지와 독서실의 거리 파악을 못함) 정성적 평가에 의존해야함

-> 정성적 평가의 리스크 존재: GPT-4o가 세상의 모든 노래를 알진 못함
1. 지식의 한계, GPT가 모르는 최신곡이나 인디 음악이 나오면 "모르겠지만 제목만 보니 어울릴 것 같네요"라고 대충 채점할 수도 있음
2. 주관적 편향, 힙합은 시끄러우니까 독서실에 안 돼"라고 편견을 가질 수 있음. (Lofi Hip-hop은 되는데)

-> 단일 지표에 의존하지 않고 '하이브리드 검증(Hybrid Validation)' 시스템을 구축
1. LLM Judge를 통해 노래의 '문화적 맥락과 분위기'를 1차로 평가하고,
2. 실제 Spotify API에서 추출한 '오디오 데이터(BPM, Energy)'로 2차 '객관적 수치 검증'을 수행.


Guardrail KPI: 치명적 오류율, 상황과 정반대되는 노래 추천 발생율 0% 유지

In [6]:
# ===========================================================
# 6. [Add-on] Guardrail KPI 심층 분석 (치명적 오류 검증)
# ===========================================================

def analyze_guardrail_kpi(df):
    """
    Guardrail KPI: 상황과 정반대되는 추천(Score 1) 발생률 0% 유지 여부 검증
    """
    total_cases = len(df)
    # LLM Score가 1점인 행만 필터링 (치명적 오류)
    fatal_errors_df = df[df['LLM Score'] == 1]
    fatal_count = len(fatal_errors_df)
    
    # 오류율 계산 (LaTeX 수식: Fatal Error Rate = (Count / Total) * 100)
    error_rate = (fatal_count / total_cases) * 100
    
    print("\n" + "="*20 + " 🛡️ Guardrail KPI Report " + "="*20)
    print(f"Total Cases: {total_cases}")
    print(f"Fatal Errors (Score 1): {fatal_count}건")
    print(f"Error Rate: {error_rate:.2f}%")
    print("-" * 60)

    # KPI 달성 여부 판단
    if fatal_count == 0:
        print("✅ [PASS] Guardrail KPI 달성! 치명적인 추천 오류가 없습니다.")
        print("   (사용자에게 불쾌감을 줄 만한 엉뚱한 추천이 0건입니다.)")
    else:
        print("🚨 [FAIL] Guardrail KPI 실패! 치명적인 오류가 발견되었습니다.")
        print(f"   목표는 0%이나, 현재 {error_rate:.2f}%의 오류가 발생했습니다.")
        print("\n🔍 [오류 상세 로그 - 프롬프트 수정이 시급합니다]")
        
        for idx, row in fatal_errors_df.iterrows():
            print(f"\n[Case ID: {row['Case ID']}] {row['Scenario']}")
            print(f"❌ 추천 결과: {row['Recommended Artist']} - {row['Recommended Title']}")
            print(f"💡 LLM 지적(Reason): {row['Judge Reason']}")
            print("-" * 60)

# -----------------------------------------------------------
# 함수 실행 (위에서 만든 result_df를 넣으세요)
analyze_guardrail_kpi(result_df)


==================== 🛡️ Guardrail KPI Report ====================
Total Cases: 20
Fatal Errors (Score 1): 0건
Error Rate: 0.00%
------------------------------------------------------------
✅ [PASS] Guardrail KPI 달성! 치명적인 추천 오류가 없습니다.
   (사용자에게 불쾌감을 줄 만한 엉뚱한 추천이 0건입니다.)


오류가 발생해서 오류로그가 뜨면 프롬프트에 해당 오류 관련 규칙 추가
오류 없으면 프롬프트 배포 확정